## Azure OpenAI GPT-V Target Demo
This notebook demonstrates how to use the Azure OpenAI GPT-V target to accept multimodal input (text+image) and generate text output.

In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.


from pyrit.models import PromptRequestPiece, PromptRequestResponse
from pyrit.prompt_target import AzureOpenAIGPTVChatTarget
from pyrit.common import default_values
import pathlib
from pyrit.common.path import HOME_PATH
import uuid

default_values.load_default_env()
test_conversation_id = str(uuid.uuid4())

# use the image from our docs
image_path = pathlib.Path(HOME_PATH) / "assets" / "pyrit_architecture.png"

request_pieces = [
PromptRequestPiece(
    role="user",
    conversation_id=test_conversation_id,
    original_prompt_text="Describe this picture:",
    original_prompt_data_type="text",
    converted_prompt_data_type="text"
), 
PromptRequestPiece(
    role="user",
    conversation_id=test_conversation_id,
    original_prompt_text=str(image_path),
    original_prompt_data_type="image_path",
    converted_prompt_data_type="image_path"
)]

In [2]:
prompt_request_response = PromptRequestResponse(request_pieces=request_pieces)

In [3]:
with AzureOpenAIGPTVChatTarget() as azure_openai_chat_target:
    resp = await azure_openai_chat_target.send_prompt_async(prompt_request=prompt_request_response) # type: ignore
    print(resp)

None: assistant: This picture is a list of components for PyRIT, with two columns labeled "Interface" and "Implementation." There are several categories listed under Interface: Target, Datasets, Scoring Engine, Attack Strategy, and Memory. Each category has a corresponding implementation listed on the right side.

For Target, there is a Local option which includes local models like ONNX, and a Remote option that involves using an API or web app.

Datasets can be either Static (prompts) or Dynamic (prompt templates).

The Scoring Engine implementation is PyRIT Itself: Self Evaluation in addition to using APIs from existing content classifiers.

Attack Strategy can be either Single Turn with static prompts or Multi Turn with multiple conversations using prompt templates.

Memory includes Storage options like JSON and Database as well as Utils for conversation, retrieval and storage, memory sharing, and data analysis.
